In [1]:
from diffusers import DiffusionPipeline
import torch
import gc

# Free up GPU memory before loading the model
torch.cuda.empty_cache()
gc.collect()

# Load Comic Diffusion Model with Optimized Settings
model_id = "ogkalu/Comic-Diffusion"

pipe = DiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Use FP16 for lower memory
    safety_checker=None,  # Disable safety checker for performance (Use responsibly)
).to("cuda")

# Optimize for Low VRAM
pipe.enable_attention_slicing()
pipe.enable_sequential_cpu_offload()

print(" Model loaded successfully!")

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu121)
    Python  3.11.9 (you have 3.11.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "C:\Users\amitm\AppData\Local\Programs\Python\Python311\Lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
    ^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'triton'


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

An error occurred while trying to fetch C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\unet: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\vae: Error no file named diffusion_pytorch_model.safetensors found in directory C:\Users\amitm\.cache\huggingface\hub\models--ogkalu--Comic-Diffusion\snapshots\ff684f581ab24e094e2055d9422e9ee076d139a8\vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.

 Model loaded successfully!


In [2]:
#  Get user input
movie_description = input("🎬 Enter a 25+ word movie description: ")

🎬 Enter a 25+ word movie description:  Capt. Jack Sparrow arrives at Port Royal in the Caribbean without a ship or crew. His timing is inopportune, however, because later that evening the town is besieged by a pirate ship. The pirates kidnap the governor's daughter, Elizabeth, who's in possession of a valuable coin that is linked to a curse that has transformed the pirates into the undead. A gallant blacksmith in love with Elizabeth allies with Sparrow in pursuit of the pirates.


In [3]:
import time

# Generate Poster
start_time = time.time()

poster = pipe(
    movie_description, 
    width=768,  # Higher quality resolution
    height=1024, 
    num_inference_steps=50  # High-quality generation
).images[0]

# Save & Display Poster
poster_path = "movie_poster.png"
poster.save(poster_path)
poster.show()

print(f"Movie Poster generated in {time.time() - start_time:.2f} sec")

Token indices sequence length is longer than the specified maximum sequence length for this model (94 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['gallant blacksmith in love with elizabeth allies with sparrow in pursuit of the pirates .']


  0%|          | 0/50 [00:00<?, ?it/s]

Movie Poster generated in 143.06 sec


In [4]:
from gpt4all import GPT4All

# Download & Load the Model (Replace with a valid model name)
gpt = GPT4All("mistral-7b-openorca.Q4_0.gguf", allow_download=True)

# Verify installation
print("GPT4All Model Loaded Successfully!")

GPT4All Model Loaded Successfully!


In [6]:
from gpt4all import GPT4All
from PIL import Image, ImageDraw, ImageFont

# Load GPT4All Model
gpt = GPT4All("mistral-7b-openorca.Q4_0.gguf")

# Generate a Movie Title & Tagline
prompt = f"Generate a compelling movie title and tagline for this plot: {movie_description}\n\nTitle: "
response = gpt.generate(prompt)

# Extract Title & Tagline
lines = response.strip().split("\n")
movie_title = lines[0].replace("Title:", "").strip() if len(lines) > 0 else "Unknown Title"
tagline = lines[1].replace("Tagline:", "").strip() if len(lines) > 1 else "Unknown Tagline"

print(f"🎬Generated Movie Title: {movie_title}")
print(f"🎬Generated Tagline: {tagline}")

# Load the Generated Movie Poster
poster_path = "movie_poster.png"  # Path to the generated poster
poster = Image.open(poster_path)

# Convert Image to Editable Format
draw = ImageDraw.Draw(poster)
width, height = poster.size

# Load Font (Uses default if arial.ttf not found)
try:
    title_font = ImageFont.truetype("arial.ttf", 80)  # Large font for title
    tagline_font = ImageFont.truetype("arial.ttf", 40)  # Smaller font for tagline
except:
    title_font = ImageFont.load_default()
    tagline_font = ImageFont.load_default()

title_x = width // 2
title_y = int(height * 0.1) 

tagline_x = width // 2
tagline_y = int(height * 0.85)  

# Add AI-Generated Title & Tagline
draw.text((title_x, title_y), movie_title, font=title_font, fill="white", anchor="mm")
draw.text((tagline_x, tagline_y), tagline, font=tagline_font, fill="white", anchor="mm")

# Save Final Poster with AI-Generated Text
output_path = "final_movie_poster.png"
poster.save(output_path)

print(f"Movie Poster saved as {output_path}")
poster.show()  # Display final poster

🎬Generated Movie Title: "Curse of the Undead Pirate"
🎬Generated Tagline: "All Hands on Deck for an Unholy Voyage!"
Movie Poster saved as final_movie_poster.png


In [ ]:

from flask import Flask, render_template, request, jsonify
import openai

app = Flask(__name__)

# Replace with your API key or model integration method
OPENAI_API_KEY = "your-api-key-here"

def generate_image(prompt):
    response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    return response["data"][0]["url"]

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        prompt = request.form["prompt"]
        image_url = generate_image(prompt)
        return render_template("index.html", image_url=image_url, prompt=prompt)
    return render_template("index.html", image_url=None)

if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())


In [1]:
from diffusers import StableDiffusionPipeline
import torch

# Model ID (Stable Diffusion v1.4)
model_id = "CompVis/stable-diffusion-v1-4"

# Load the model (will download if not found)
pipe = StableDiffusionPipeline.from_pretrained(model_id)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

print("✅ Stable Diffusion is ready!")


WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu121)
    Python  3.11.9 (you have 3.11.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "C:\Users\amitm\AppData\Local\Programs\Python\Python311\Lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
    ^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'triton'


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:  29%|##9       | 357M/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   3%|3         | 115M/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:  44%|####3     | 147M/335M [00:00<?, ?B/s]

model.safetensors:  94%|#########3| 461M/492M [00:00<?, ?B/s]

C:\Users\amitm\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amitm\.cache\huggingface\hub\models--CompVis--stable-diffusion-v1-4. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Stable Diffusion is ready!
